<a href="https://colab.research.google.com/github/saranyapalani/sample/blob/master/rough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
from matplotlib.colors import rgb2hex
from matplotlib.cm import get_cmap
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
import plotly.offline as py
# from __future__ import absolute_import, division, print_function, unicode_literals

from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM, CuDNNLSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import ConvLSTM2D
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [0]:
df = pd.read_csv('/content/drive/My Drive/dataset.csv',error_bad_lines=False)
df.head()
df = df.fillna(method='ffill')

In [0]:
original_columns = df.columns
columns2keep = ['ID','Date','Type','Location Description','Arrest','Year']
df = df[columns2keep]
df.head(2)

,ID,Date,Type,Location Description,Arrest,Year
0,10508693,05-03-2017 23:40,ROAD ACCIDENTS,APARTMENT,True,2017
1,10508695,05-03-2017 21:40,ROAD ACCIDENTS,RESIDENCE,False,2017


In [0]:
df['Date'] = pd.to_datetime(df['Date'].astype(str), errors='coerce')
df['Date'] = pd.to_datetime(df['Date'], format = '%d/%m/%Y')
df.head()


,ID,Date,Type,Location Description,Arrest,Year
0,10508693,2017-05-03 23:40:00,ROAD ACCIDENTS,APARTMENT,True,2017
1,10508695,2017-05-03 21:40:00,ROAD ACCIDENTS,RESIDENCE,False,2017
2,10508697,2017-05-03 23:31:00,PUBLIC PEACE VIOLATION,STREET,False,2017
3,10508698,2017-05-03 22:10:00,ROAD ACCIDENTS,SIDEWALK,False,2017
4,10508699,2017-05-03 22:00:00,JEWELS THEFT AND MURDER,RESIDENCE,False,2017


In [0]:
#df['Date'] = pd.to_datetime(df['Date'])
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['hour']=df['Date'].dt.hour


In [0]:
df.head()

,ID,Date,Type,Location Description,Arrest,Year,month,day,hour
0,10508693,2017-05-03 23:40:00,ROAD ACCIDENTS,APARTMENT,True,2017,5.0,3.0,23.0
1,10508695,2017-05-03 21:40:00,ROAD ACCIDENTS,RESIDENCE,False,2017,5.0,3.0,21.0
2,10508697,2017-05-03 23:31:00,PUBLIC PEACE VIOLATION,STREET,False,2017,5.0,3.0,23.0
3,10508698,2017-05-03 22:10:00,ROAD ACCIDENTS,SIDEWALK,False,2017,5.0,3.0,22.0
4,10508699,2017-05-03 22:00:00,JEWELS THEFT AND MURDER,RESIDENCE,False,2017,5.0,3.0,22.0


In [0]:
df['month'] = df.month.astype('Int64')
df['day'] = df.day.astype('Int64')
df['hour'] = df.hour.astype('Int64')



In [0]:
df.head()

,ID,Date,Type,Location Description,Arrest,Year,month,day,hour
0,10508693,2017-05-03 23:40:00,ROAD ACCIDENTS,APARTMENT,True,2017,5,3,23
1,10508695,2017-05-03 21:40:00,ROAD ACCIDENTS,RESIDENCE,False,2017,5,3,21
2,10508697,2017-05-03 23:31:00,PUBLIC PEACE VIOLATION,STREET,False,2017,5,3,23
3,10508698,2017-05-03 22:10:00,ROAD ACCIDENTS,SIDEWALK,False,2017,5,3,22
4,10508699,2017-05-03 22:00:00,JEWELS THEFT AND MURDER,RESIDENCE,False,2017,5,3,22


In [0]:
groups = df.groupby(['Year', 'month', 'day'])

# The goal is to check when the day as well as the month are changing.
df['day_change'] = np.append(0, np.diff(df.day)) # We need to add the last value
df['month_change'] = np.append(0, np.diff(df.month)) # We need to add the last value
df[['month_change', 'day_change']]

df['day'] = 1
n_day = []  # List containing the day numbers of each row in the dataframe
for index in range(len(df)):
    if index == 0:
        n_day.append(1)
 #   elif df['month_change'].values[index] != 0:
      #  n_day.append(1) # If month changing we start back at the 1st of the month
    elif df['day_change'].values[index] != 0:
        n_day.append(n_day[-1] +1 ) # If the day is changing we increment the number
    else:
        n_day.append(n_day[-1])  # If not we keep the same day number 
df['day'] = np.array(n_day, dtype=np.int)

In [0]:
#groups = df.groupby(['Year', 'month', 'day','hour'])
df.AccidentHour.isna().sum()
df['Year'] = df.Year.astype('str')
df['Month'] = df.month.astype('str')
df['Day'] = df.day.astype('str')
df['AccidentHour'] = df.hour.astype('str')
#df['Hour'] = df.AccidentHour.astype('str')

df['timestamp'] = df.Year + ' ' + df.month + ' ' + df.day + ' ' + df.hour
df['timestamp'] = df['timestamp'].apply(lambda x: datetime.strptime(x, '%Y %m %d %H'))

df = df.set_index('timestamp')

TypeError: ignored

In [0]:
def get_df_count(df, dt=None, en_plot=False):
    if dt is not None:
        df = df.reset_index()
        df.Date =  df.Date.dt.round(dt)
        df = df.set_index('Date')
    df_count = pd.DataFrame()
    df_count['Type'] = df.groupby('Date').count()
    if en_plot:
        plt.figure(figsize=(15, 12))
        plt.plot(df_count['Type'])
        plt.title('Number of accident per timestamp in the dataset')
    return df_count

df_count = get_df_count(df, dt='1D', en_plot=True)
df_count.describe()


ValueError: ignored